# This is a separte tool call module i created for understanding Tool calls manually

In [40]:
import gradio as gr
from dotenv import load_dotenv


In [41]:
load_dotenv(override=True)

True

In [51]:
Bviews=0
Cviews=0


In [59]:
def Channel1():
    global Bviews
    Bviews +=1
    print("Welcome to Bollywood Crazy -- Your ultimate source of Bollywod related news , box office and many more. Please do like and subsribe the content")
    print(f"Currently viewer {Bviews}")    
    return f" views - {Bviews}"

In [60]:
def Channel2():
    global Cviews
    print("Cricket Mania -Your ultimate Cricket chhannel. Get latest cricketing news around the world ")
    Cviews +=1
    print(f"Currently views {Cviews}")
    return f" views - {Cviews}"

In [26]:
def unknownQuestion():
    sPrompt="You are not familiar with subject . Reply stating you are Not aware of the question. Direct user to ask question related to cricket and bollywood only"
    return "I am Not familair with the subject . I cannot answer"


In [27]:
Channel1_json={
    "name":"Channel1",
    "description":"Yotube Channel related to Bollywood matters"

}

In [28]:
Channel2_json={
    "name":"Channel2",
    "description":"Yotube Channel related to Cricketing matters"

}

In [29]:
unknownQuestion_json={
    "name":"unknownQuestion",
    "description":"This is UNKNOWN TOPIC , NOT COVERED BY ANY CHHANEL"
}

In [13]:
name="kiran_das"

In [31]:
tools=[{"type":"function","function":Channel1_json},
        {"type":"function","function":Channel2_json},
        {"type":"function","function":unknownQuestion_json}]

In [32]:
tools

[{'type': 'function',
  'function': {'name': 'Channel1',
   'description': 'Yotube Channel related to Bollywood matters'}},
 {'type': 'function',
  'function': {'name': 'Channel2',
   'description': 'Yotube Channel related to Cricketing matters'}},
 {'type': 'function',
  'function': {'name': 'unknownQuestion',
   'description': 'This is UNKNOWN TOPIC , NOT COVERED BY ANY CHHANEL'}}]

In [57]:
SystemPrompt=f"You are {name}. Introduce first with your Chhanel for which the question is asked .You have Two channels ,one Related to Bollywood and other Related to Cricket"
SystemPrompt+=f"whenever user ask you anything based on subject you contact either of the tools"
SystemPrompt+="if User wants to know about Bollywood you use the Channel1 tool to answer the question related to the subject.You are an expert in Bollywood related matter"
SystemPrompt+="Any question related to Bollywood , you can answer properly. This chhanel only provides bollywood related info "
SystemPrompt+="if User wants to know about Cricket related matter use the Channel2 tool to answer the question related to the subject.You are an expert in Cricketing related matter. Any question related top cricket , you can answer properly. This chhanel only provides cricket related news"
SystemPrompt+="if User ask anything aprt from Bollywood or Cricket use the unknownQuestion tool to answer the question related to the subject "
SystemPrompt+="Based on the above provide a short concised answer"

In [35]:
import json

In [44]:
# This is a more elegant way that avoids the IF statement.

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [37]:
from openai import OpenAI
openai=OpenAI()

In [38]:
def chat(message,history):
    messages=[{"role":"system","content":SystemPrompt}] + history +[{"role":"user","content":message}]
    done=False
    while not done:
        response=openai.chat.completions.create(model="gpt-4o-mini",messages=messages,tools=tools)
        finish_reason=response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message=response.choices[0].message
            print(f"-----message :{message}")
            tool_calls=message.tool_calls
            result=handle_tool_calls(tool_calls=tool_calls)
            messages.append(message)
            messages.extend(result)
        else:
            done=True

    return response.choices[0].message.content

In [58]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


-----message :ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_waaBStqSKYVgs8Z1DjGbzwWA', function=Function(arguments='{}', name='unknownQuestion'), type='function')])
Tool called: unknownQuestion
-----message :ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ZfiMkZ7PCPtIfmTSwVTlQsqM', function=Function(arguments='{}', name='Channel2'), type='function')])
Tool called: Channel2
Cricket Mania -Your ultimate Cricket chhannel. Get latest cricketing news around the world 
-----message :ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oCDNVhh4VXDFMlDFW3IyacGZ', function=Function(arguments='{}', name='Channel2'), type='function')])
Tool called: 

# below is genrted by the AI 

In [61]:
# Custom Gradio UI showing Bviews and Cviews counters
import gradio as gr

# Wrapper to adapt Chatbot (tuples) history to your chat() function
# and to surface the current Bviews/Cviews values in the UI

def chat_and_counts(message, history):
    # Convert tuple-based chatbot history to message list for chat()
    converted_history = []
    if history:
        for user_msg, bot_msg in history:
            if user_msg is not None:
                converted_history.append({"role": "user", "content": user_msg})
            if bot_msg is not None:
                converted_history.append({"role": "assistant", "content": bot_msg})

    reply = chat(message, converted_history)

    # Read globals to show counters
    try:
        b = Bviews
    except NameError:
        b = 0
    try:
        c = Cviews
    except NameError:
        c = 0

    updated_history = history + [(message, reply)]
    return updated_history, f"B views: {b}", f"C views: {c}"

with gr.Blocks() as demo:
    gr.Markdown("### Manual Tool Call Chat with View Counters")
    with gr.Row():
        chatbot = gr.Chatbot(label="Chat", show_label=True)
        with gr.Column():
            b_count = gr.Markdown("B views: 0")
            c_count = gr.Markdown("C views: 0")
    with gr.Row():
        msg = gr.Textbox(label="Message", placeholder="Type your message...")
        send = gr.Button("Send", variant="primary")

    send.click(fn=chat_and_counts, inputs=[msg, chatbot], outputs=[chatbot, b_count, c_count])
    msg.submit(fn=chat_and_counts, inputs=[msg, chatbot], outputs=[chatbot, b_count, c_count])

# Launch this UI instead of the basic ChatInterface if desired
# Comment out the following line if you still want to use the original ChatInterface launch above
# and run this cell separately when you want counters in the UI.
demo.launch()


/var/folders/nc/2bjvclms3k13mrldbxsjxx780000gn/T/ipykernel_15932/3919105030.py:35: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chat", show_label=True)


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


-----message :ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9HsXGPFNe6q7tti8dYlWAnkS', function=Function(arguments='{}', name='Channel1'), type='function')])
Tool called: Channel1
Welcome to Bollywood Crazy -- Your ultimate source of Bollywod related news , box office and many more. Please do like and subsribe the content
Currently viewer 3
-----message :ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_tn8CthABLjLN1mN5aocmCTUv', function=Function(arguments='{}', name='Channel1'), type='function')])
Tool called: Channel1
Welcome to Bollywood Crazy -- Your ultimate source of Bollywod related news , box office and many more. Please do like and subsribe the content
Currently viewer 4
-----message :ChatCompletionMessage(content=None, refusal=None, role='as